# basicsynbio Addgene submission collection

This Notebook contains code to aid the submission of basicsynbio linkers to addgene.

## Notes on data preperation

- [x] The p15A plasmid supplied by Marko `BS_x6x` was corrected given it has a "t" insertion.
- [x] The Benchling feature library (BASIC_SEVA_benchling_misc_features_library.csv) was updated commit: `a2fb18c`.

## Aims and objectives for cell/s below

- [x] Import the following objects:
  - [x] pickled BasicLinker `BSEVA_L1`
  - [x] Selected Oris and AbR markers specified in #130.
  - [x] mScarlet counter selection casettes for associated Oris.
- [x] Assemble BS_x7x_TS gblocks with BS_1xx to place them in storage vectors. Export associated data. 
- [x] Generate a dictionary mapping between Oris and counter-selection casettes.
- [ ] Make assemblies and export the following data:
  - [x] Genbank file containing all correctly assembled backbones.
  - [x] Manual and automated build instructions.

In [1]:
import basicsynbio as bsb
from basicsynbio.utils import MARKER_DICT, ORI_DICT
from pathlib import Path
import pickle
import re

In [2]:
path_to_seqs = Path.cwd().parents[0] / "sequences"
path_to_initial_mods = path_to_seqs / "genbank_files" / "BASIC_SEVA_collection" / "initial_mods"
# Import objects
with open(path_to_seqs / "alternative_formats" / "pickles" / "SEVA-BB1", "rb") as file:
    bb_linker = pickle.load(file)
abr_markers = list(
    bsb.import_parts(path_to_initial_mods / "AbR_markers.gb",
    "genbank"
))
oris = list(
    bsb.import_parts(path_to_initial_mods / "oris.gb",
    "genbank"
))
cs_cassettes = list(
    bsb.import_parts(path_to_initial_mods / "mScarlet_cs_cassettes.gb",
    "genbank"
))
ts_ori_gblocks = list(bsb.import_parts(path_to_initial_mods / "TS_ori_gblocks.gb", "genbank"))

In [3]:
# Assemble BS_x7x_TS
linkers = bsb.BASIC_BIOLEGIO_LINKERS["v0.1"]
ts_ori_assemblies = [bsb.BasicAssembly(
    ts_ori.id,
    linkers["LMP"],
    ts_ori,
    linkers["LMS"],
    [marker for marker in abr_markers if marker.id == "BS_1xx"][0]
) for ts_ori in ts_ori_gblocks]
bsb.export_sequences_to_file(
    ts_ori_assemblies,
    path_to_initial_mods / "TS_ori_modules.gb"
)
bsb.pdf_instructions(
    bsb.BasicBuild(*ts_ori_assemblies),
    "/home/hainesm6/github_repos/LondonBiofoundry/basicsynbio/pdfs/bs_x7x_ts_instructions.pdf",
)

/home/hainesm6/.pyenv/versions/3.8.12/envs/basicsynbio/lib/python3.8/site-packages/Bio/SeqIO/InsdcIO.py:721: BiopythonWarning: Increasing length of locus line to allow long name. This will result in fields that are not in usual positions.
  warnings.warn(


[BasicPart(seq=Seq('TCTGGTGGGTCTCTGTCCTGTTCTACTTTTGTTTGTTAGTCTTGATGCTTCACT...TCG'), id='BS_x7x_TS_CASc', name='BS_x7x_TS_CASc', description='', dbxrefs=[]),
 BasicPart(seq=Seq('TCTGGTGGGTCTCTGTCCTGTTCTACTTTTGTTTGTTAGTCTTGATGCTTCACT...TCG'), id='BS_x7x_TS_PKD46', name='BS_x7x_TS_PKD46', description='', dbxrefs=[])]

In [4]:
# Map oris with counter selection cassettes
ori_cs_cassette_mapping = {
    "BS_x6x": "B407_J23119-RBS34-mScarl",
    "BS_x7x": "B408_J23119-RBS-A12-mSc",
    "BS_x7x_pKD46": "B408_J23119-RBS-A12-mSc",
    "BS_x9x": "B407_J23119-RBS34-mScarl",
}
ori_cs_cassette_mapping = {
    key: cs_cassettes[[cs_cassette.id for cs_cassette in cs_cassettes].index(value)] for key, value in ori_cs_cassette_mapping.items()
}
# Make assemblies and export data
MARKER_DICT = MARKER_DICT.copy()
MARKER_DICT["5a"] = "Tetracycline"
assemblies = []
for abr_marker in abr_markers:
    if re.match("BS_\daxx", abr_marker) != None:
        abr_abreviation = abr_marker.id[-4:-2]
    else:
        abr_abreviation = abr_marker.id[-3]
    for ori in oris:    
        assembly = bsb.BasicAssembly(
            "foobar",
            linkers["LMP"],
            abr_marker,
            bb_linker,
            ori,
            linkers["LMS"],
            ori_cs_cassette_mapping[ori.id],
        )
        if ori.id == "BS_x7x_pKD46":
            assembly.id = f"BASIC_SEVA_{abr_abreviation}7_pKD46.10"
            assembly.description = f"BASIC SEVA vector containing {MARKER_DICT[abr_abreviation]} resistance marker and temperature sensitive pSC101 origin of replication."
        else:
            assert re.match("BS_x\dx", ori.id) != None
            assembly.id = f"BASIC_SEVA_{abr_abreviation + ori.id[-2]}.10"
            assembly.description = f"BASIC SEVA vector containing {MARKER_DICT[abr_abreviation]} resistance marker and {ORI_DICT[ori.id[-2]]} origin of replication."
        assemblies.append(assembly)
build = bsb.BasicBuild(*assemblies)
bsb.export_sequences_to_file(
    (assembly.return_part(description=assembly.description, name=assembly.id) for assembly in build.basic_assemblies),
    Path.cwd().parents[0] / "basicsynbio" / "parts_linkers" / "BASIC_SEVA_collection_v10.gb"
)
bsb.export_echo_assembly(
    build,
    Path.cwd().parents[0] / "csv_xlsx_files" / "BASIC_SEVA_collection_v10_echo_scripts.zip",
    assemblies_per_clip=20,
    alternate_well=True,
)
bsb.pdf_instructions(
    build,
    "/home/hainesm6/github_repos/LondonBiofoundry/basicsynbio/pdfs/BASIC_SEVA_collection_v10_manual.pdf",
    assemblies_per_clip=20,
)


NameError: name 'oris' is not defined